## Parte 3: Construyendo un Sistema de Recomendacion con Feedback Implicito





### Paso 1: Agregando las Librerias

Estos seran las librerias que utilizaremos:

- [numpy](https://numpy.org/)
- [pandas](https://pandas.pydata.org/)
- [implicit](https://github.com/benfred/implicit)
- scipy (en especifico la clase **csr_matrix**)

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

import implicit

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Paso 2: Cargando los datos



In [2]:
import os
import json
DIR = "/Users/juan_arturo_rg/Documents/Big Data/Proyecto/"
os.chdir(DIR)

#movies = pd.read_json("australian_users_items.json")
#movies.head()

from datasets import load_dataset

dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_All_Beauty")

products = pd.DataFrame(dataset["full"])

dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_All_Beauty")

reviews = pd.DataFrame(dataset["full"])
reviews

#https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023
#https://cseweb.ucsd.edu/~jmcauley/datasets.html#amazon_reviews
#https://github.com/hyp1231/AmazonReviews2023/tree/main/amazon-c4



Found cached dataset amazon-reviews-2023 (/Users/juan_arturo_rg/.cache/huggingface/datasets/McAuley-Lab___amazon-reviews-2023/raw_meta_All_Beauty/0.0.0/16b76e0823d73bb8cff1e9c5e3e37dbc46ae3daee380417ae141f5e67d3ea8e8)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset amazon-reviews-2023 (/Users/juan_arturo_rg/.cache/huggingface/datasets/McAuley-Lab___amazon-reviews-2023/raw_review_All_Beauty/0.0.0/16b76e0823d73bb8cff1e9c5e3e37dbc46ae3daee380417ae141f5e67d3ea8e8)


  0%|          | 0/1 [00:00<?, ?it/s]

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5.0,Such a lovely scent but not overpowering.,This spray is really nice. It smells really go...,[],B00YQ6X8EO,B00YQ6X8EO,AGKHLEW2SOWHNMFQIJGBECAF7INQ,1588687728923,0,True
1,4.0,Works great but smells a little weird.,"This product does what I need it to do, I just...",[],B081TJ8YS3,B081TJ8YS3,AGKHLEW2SOWHNMFQIJGBECAF7INQ,1588615855070,1,True
2,5.0,Yes!,"Smells good, feels great!",[],B07PNNCSP9,B097R46CSY,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,1589665266052,2,True
3,1.0,Synthetic feeling,Felt synthetic,[],B09JS339BZ,B09JS339BZ,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,1643393630220,0,True
4,5.0,A+,Love it,[],B08BZ63GMJ,B08BZ63GMJ,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,1609322563534,0,True
...,...,...,...,...,...,...,...,...,...,...
701523,4.0,Four Stars,Conditioner is great shampoo not as I expected,[],B006YUIWKA,B006YUIWKA,AFIXGFVEGLMOTMBTJL7H3VSIETDQ,1478227021000,0,True
701524,1.0,Pretty,Did not work! Used the whole bottle and my hai...,[],B006YUIWKA,B006YUIWKA,AFV7YZFOJF564EZGET5LG45K4QEA,1480908730000,0,False
701525,5.0,Great sunless tanner,Product as expected. Shipping was on time.,[],B06ZZV9MZT,B06ZZV9MZT,AHYDCWDMMVMLBX7FY7M7JKADKRDQ,1590547974067,0,True
701526,5.0,The Crown on top is a Ring!!!,"Not only is it a delicious fragrance, but also...",[],B000HB6VLE,B000HB6VLE,AF6ZIAEN7TQ2WY5ZL77F6JDPV7XQ,1184798209000,4,False


In [3]:
l = reviews.groupby(['user_id'], as_index=False).count()

reviews = reviews[reviews['user_id'].isin(l[l.rating > 2].user_id.unique())]

reviews = reviews[reviews['parent_asin'].isin(products['parent_asin'].unique())]

products = products[products['parent_asin'].isin(reviews['parent_asin'].unique())]

In [4]:
reviews

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
8,5.0,Great for at home use and so easy to use!,This is perfect for my between salon visits. I...,[],B08P2DZB4X,B08P2DZB4X,AFSKPY37N3C43SOI5IEXEK5JSIYA,1627391044559,0,False
9,5.0,Nice shampoo for the money,I get Keratin treatments at the salon at least...,[],B086QY6T7N,B086QY6T7N,AFSKPY37N3C43SOI5IEXEK5JSIYA,1626614511145,0,False
10,3.0,Not what I thought I would be getting,I was very disappointed when I got this facial...,[],B08DHTJ25J,B08DHTJ25J,AFSKPY37N3C43SOI5IEXEK5JSIYA,1626211245370,0,False
11,5.0,A little goes a long way!,This is a really nice moisturizing lotion. It ...,[],B07RBSLNFR,B07RBSLNFR,AFSKPY37N3C43SOI5IEXEK5JSIYA,1621184430697,0,False
12,3.0,Just ok,I try to get Keratin treatments every 3 months...,[],B07SLFWZKN,B07SLFWZKN,AFSKPY37N3C43SOI5IEXEK5JSIYA,1619737501209,0,False
...,...,...,...,...,...,...,...,...,...,...
700663,5.0,Water waves on swim !!,[[VIDEOID:1552f5dc4ee822d5d7a90ac9288ba72c]] O...,"[{'attachment_type': 'IMAGE', 'large_image_url...",B07NXQYVMH,B07NXQYVMH,AGCESJO2YAHFW3GKOY2JSS5QC6DA,1556813138288,19,True
700664,5.0,Silk silk is that you ??!!,"Omg,omg, omg!!! Want silky hair? Look no furth...","[{'attachment_type': 'IMAGE', 'large_image_url...",B071RVQQBD,B071RVQQBD,AGCESJO2YAHFW3GKOY2JSS5QC6DA,1537841089116,0,True
700823,5.0,Classic,Classic black polish with not much more to tel...,[],B005P4AIZ8,B09P3ZMV55,AGNEYGHRA2MM5MUN6WCZRXDSNJZA,1403425900000,1,False
700824,2.0,Not really what it looks like here,On the picture it looks like milky half transp...,[],B005P49I7W,B09P3ZMV55,AGNEYGHRA2MM5MUN6WCZRXDSNJZA,1403425754000,0,True


In [5]:
products

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
10,All Beauty,Avon Big & False Lash Mascara - Brown/Black by...,4.0,7,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",AVON,[],"{""Color"": ""black"", ""Brand"": ""AVON"", ""Item Volu...",B01555WAOS,None,None,None
13,All Beauty,"Loose Powder, 1.5oz Bottle Loose Face Powder, ...",4.2,22,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",GETMORE7,[],{},B07FVZVQKV,None,None,None
17,All Beauty,PPY Eyelash Growth Serum – Natural Ingredients...,3.9,114,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",PPY,[],"{""Use for"": ""Face"", ""Scent"": ""Aloe"", ""Brand"": ...",B07WFSQXL5,None,None,None
19,All Beauty,Nadula Hair 10A Brazilian Body Wave U Part Hum...,3.6,11,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Nadula,[],"{""Material"": ""Human Hair"", ""Hair Type"": ""Curly...",B07MH4Z7J7,None,None,None
22,All Beauty,VIROCHEMISTRY Pheromones For Women (Elixir) - ...,3.7,33,[🗽 SCIENTIFICALLY PROVEN! Scientifically Formu...,[The Most Amazingly Effective and Fantastic Sm...,29.8,{'hi_res': ['https://m.media-amazon.com/images...,{'title': ['VIROCHEMISTRY Pheromone Perfume To...,VIROCHEMISTRY,[],"{""Brand"": ""VIROCHEMISTRY"", ""Item Form"": ""Liqui...",B07X1TK3VS,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112462,All Beauty,Utips Hair Extensions Human Hair Black Ombre G...,3.7,7,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Sunny Hair,[],"{""Brand"": ""Sunny Hair"", ""Material"": ""Human Hai...",B06XDMWDC2,None,None,None
112478,All Beauty,Ellipse Flat Eyelash Extensions 0.15mm Curl D ...,3.1,4,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",QUEWEL,[],"{""Package Dimensions"": ""4.65 x 2.44 x 0.51 inc...",B07SQVP187,None,None,None
112495,All Beauty,Play Tailor Kids Haircut Cape Barber Apron for...,4.3,12,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Green Hills,[],{},B08V4GZGKY,None,None,None
112512,All Beauty,Brazilian Virgin Hair Lace Front Wig Remy Huma...,5.0,1,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",None,[],"{""Is Discontinued By Manufacturer"": ""No"", ""Pac...",B01KFCXKIG,None,None,None


Revisamos el contenido de ratings

### Paso 3: Transformando los datos



In [6]:
def create_X(df: pd.DataFrame):
    """
    Generates a sparse matrix from ratings dataframe.

    Args:
        df: pandas dataframe

    Returns:
        X: sparse matrix
        user_mapper: dict that maps user id's to user indices
        user_inv_mapper: dict that maps user indices to user id's
        movie_mapper: dict that maps movie id's to movie indices
        movie_inv_mapper: dict that maps movie indices to movie id's
    """
    N = df['user_id'].nunique()
    M = df['parent_asin'].nunique()

    user_mapper = dict(zip(np.unique(df["user_id"]), list(range(N))))
    product_mapper = dict(zip(np.unique(df["parent_asin"]), list(range(M))))

    user_inv_mapper = dict(zip(list(range(N)), np.unique(df["user_id"])))
    product_inv_mapper = dict(zip(list(range(M)), np.unique(df["parent_asin"])))

    user_index = [user_mapper[i] for i in df['user_id']]
    product_index = [product_mapper[i] for i in df['parent_asin']]

    X = csr_matrix((df["rating"], (product_index, user_index)), shape=(M, N))

    return X, user_mapper, product_mapper, user_inv_mapper, product_inv_mapper

In [7]:
X, user_mapper, product_mapper, product_inv_mapper, product_inv_mapper = create_X(reviews)

### Creando los Mapeos de los titulos de los productos


In [8]:
from fuzzywuzzy import process

def product_finder(title):
    all_titles = products['title'].tolist()
    closest_match = process.extractOne(title,all_titles)
    return closest_match[0]

product_title_mapper = dict(zip(products['title'], products['parent_asin']))
product_title_inv_mapper = dict(zip(products['parent_asin'], products['title']))

def get_product_index(title):
    fuzzy_title = product_finder(title)
    product_id = product_title_mapper[fuzzy_title]
    product_idx = product_mapper[product_id]
    return product_idx

def get_product_title(product_idx):
    product_id = product_inv_mapper[product_idx]
    title = product_title_inv_mapper[product_id]
    return title

/Users/juan_arturo_rg/anaconda3/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### Paso 4: Construyendo el modelo de modelo de Recomendacion de Feedback Implicito



In [9]:
model = implicit.als.AlternatingLeastSquares(factors=50)


/Users/juan_arturo_rg/anaconda3/lib/python3.11/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 12 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


In [10]:
model.fit(X.T.tocsr())

  0%|          | 0/15 [00:00<?, ?it/s]

In [11]:
product_of_interest = 'shampoo'

product_index = get_product_index(product_of_interest)
related = model.similar_items(product_index)
related

(array([ 7108,  7702,  7128,  6407, 19182,  4699,  4163,  4210,  3878,
         3081], dtype=int32),
 array([0.9999998 , 0.953707  , 0.94895655, 0.94878125, 0.94224215,
        0.9304286 , 0.9274732 , 0.92747307, 0.92747307, 0.92747307],
       dtype=float32))

In [12]:
print(f"Por que miraste compraste {product_finder(product_of_interest)} te pueden interesar:")
for t, r in zip(related[0], related[1]):

    recommended_title = get_product_title(t)
    if recommended_title != product_finder(product_of_interest):
        print(recommended_title)



Por que miraste compraste Shampoo te pueden interesar:
MERISIHAIR Wine Red Long Straight Hair Synthetic Wigs Heat Resistant Party Cosplay Ombre Color for Women
Short Bob Straight Wigs Synthetic Lace Front Wigs Glueless For Black Women Natural Black 14"
Malaysian Curly Hair Bundles Deep Weave 100% Unprocessed Remy Kinky Curly Human Hair Bundles Hair Weft Extensions Natural Black Color By Lovenea (8" 10" 12")
BestMal Curly Headband Wig 10 Inch Deep Wave Headband Wigs for Black Women Human Hair Wavy Wigs with Headband 150% Density Half Wigs Glueless None Lace Front Wigs Natural Black
Manicure Pedicure Set Nail Clippers Cutters Stainless Steel Professional Toenail Clippers Nail Scissors Grooming Kit for Men Women,Travel Case,10pcs
Bonita Salon 6-piece Nail Polish Color Set
KINGDOMCARES New Year Gifts Electric Mask Heater Essence Facial Mask Pure Natural Face Mask for Facial Treatment Reduce Wrinkles Acne Scars Blackheads Cellulite Ultimate Spa Quality Skin Care Golden
Lula Lilly Shower Cap

### Paso 5: Generando las recomendaciones del usuario


In [13]:
user_id = 'AFSKPY37N3C43SOI5IEXEK5JSIYA'


In [14]:
l = reviews[['user_id', 'parent_asin']].groupby('user_id', as_index=False).nunique()
l[l['parent_asin'] > 1]

,user_id,parent_asin
0,AE225THXPLSO5QQ3PHQP7SPETM2A,3
2,AE22XHMBOBJBXUFCTNYLFMD4UKMA,3
3,AE23NZUELB4BYWLHKWJXAS73PTSQ,3
4,AE23ZBUF2YVBQPH2NN6F5XSA3QYQ,16
5,AE24J5HKPC7GM56SA2CW2HGF3YXQ,3
...,...,...
9153,AHZYZ2CXUVWNGQTETZVAP3NW5MOA,3
9154,AHZYZL5IDS67DNZWC3KSKFTR6NXA,5
9155,AHZZ3SRJZNF3NRKIAI32UF345O6Q,5
9156,AHZZ6SGWPYHMV7EKQDBVUTHHFZUA,3


In [15]:
user_ratings = reviews[reviews['user_id']==user_id].merge(products[['parent_asin', 'title']], on= ['parent_asin'])
user_ratings = user_ratings.sort_values('rating', ascending=False)
print(f"El numero de productos rankeados por el usuario {user_id} es de: {user_ratings['parent_asin'].nunique()}")

El numero de productos rankeados por el usuario AFSKPY37N3C43SOI5IEXEK5JSIYA es de: 15


In [16]:
user_ratings = reviews[reviews['user_id']==user_id].merge(products[['parent_asin', 'title']], on= ['parent_asin'])
user_ratings = user_ratings.sort_values('rating', ascending=False)
top_5 = user_ratings.head()
top_5

,rating,title_x,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,title_y
0,5.0,Great for at home use and so easy to use!,This is perfect for my between salon visits. I...,[],B08P2DZB4X,B08P2DZB4X,AFSKPY37N3C43SOI5IEXEK5JSIYA,1627391044559,0,False,NIRA Skincare Laser & Serum Bundle - Includes ...
1,5.0,Nice shampoo for the money,I get Keratin treatments at the salon at least...,[],B086QY6T7N,B086QY6T7N,AFSKPY37N3C43SOI5IEXEK5JSIYA,1626614511145,0,False,Caroline Keller Keratin Shampoo for dry and da...
3,5.0,A little goes a long way!,This is a really nice moisturizing lotion. It ...,[],B07RBSLNFR,B07RBSLNFR,AFSKPY37N3C43SOI5IEXEK5JSIYA,1621184430697,0,False,OGANA CELL Peptide Concentrating Amazing Lotio...
5,5.0,Good quality hair brush!,"Really nice small brush. Made well, nice wood ...",[],B08JTNQFZY,B08JTNQFZY,AFSKPY37N3C43SOI5IEXEK5JSIYA,1617904219785,0,False,GAINWELL
6,5.0,Great refreshing skin care routine!,I had never tried anything for my skin consist...,[],B08GLG6W8T,B08GLG6W8T,AFSKPY37N3C43SOI5IEXEK5JSIYA,1613319236253,0,False,Hanhoo Red pomegranate 6pcs Skin Care Set (6pc...


In [17]:
bottom_5 = user_ratings[user_ratings['rating']<5].tail()
bottom_5

,rating,title_x,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,title_y
2,3.0,Not what I thought I would be getting,I was very disappointed when I got this facial...,[],B08DHTJ25J,B08DHTJ25J,AFSKPY37N3C43SOI5IEXEK5JSIYA,1626211245370,0,False,Orange Peel Nature's Cleanse Facial Scrub - by...
4,3.0,Just ok,I try to get Keratin treatments every 3 months...,[],B07SLFWZKN,B07SLFWZKN,AFSKPY37N3C43SOI5IEXEK5JSIYA,1619737501209,0,False,Keratin Secrets Do It Yourself Home Keratin Sy...
7,3.0,Didn't do much and difficult to place and main...,"When I saw this, I was thrilled to be able to ...",[],B08M3C6LVS,B08M3C6LVS,AFSKPY37N3C43SOI5IEXEK5JSIYA,1607339460872,0,False,24K Gold Under Eye Mask– 20 Pairs- Under Eye P...
14,3.0,I just don't get it,I don't see the fuss with this toothbrush. As ...,[],B00JMDPK8S,B00JMDPK8S,AFSKPY37N3C43SOI5IEXEK5JSIYA,1404402463000,0,False,Panasonic EW-DL82 Sonic Vibration Rechargeable...


In [18]:
X_t = X.T.tocsr()
user_idx = user_mapper[user_id]
recommendations = model.recommend(user_idx, X_t[user_idx])
recommendations

(array([12912,  9688, 16864, 10813,  8747,  9586, 13029, 14250, 12338,
         5403], dtype=int32),
 array([0.38144287, 0.36304116, 0.3527121 , 0.3324364 , 0.32716575,
        0.32504803, 0.31793803, 0.31439716, 0.30985808, 0.30795956],
       dtype=float32))

In [19]:
for t, r in zip(recommendations[0], recommendations[1]):
    recommended_title = get_product_title(t)
    print(recommended_title)

Gentlehomme Face Moisturizer For Men - Mens Facial Lotion with Neroli Essential Oil Scent - Collagen & Stem Cells - Daily Anti-Aging Moisturizer for Dry & Oily Skin - 4 oz
Water Flosser Cordless Teeth Cleaner, SOOCAS Professional Oral Irrigator 230ML Portable and Rechargeable Water Dental Flosser with 3 Modes and 4 Jet Tips, IPX7 Waterproof, USB Charged for 30 Days Use
John Frieda Detox and Repair Shampoo and Conditioner Set with Nourishing Avocado Oil and Green Tea, 8.45 Fl Oz (Pack of 2)
Mountain Falls Regular Nail Polish Remover for Natural Nails with Vitamin E and Panthenol, 10 Fluid Ounce
TRESemmé Botanique Shampoo and Conditioner Four Pack, Nourish & Replenish, 88 oz
Makeup Blender Sponge Set - 4 Pack Latex Free Q-Soft Makeup Beauty Foundation Blending Sponge Flawless Smooth Brush for Liquid Foundation BB Creams Powders Concealer (4 Pack)
Caudalie Favorites Set
Wonder4 Electric Callus Remover - Rechargeable Electric Foot File Callus Shaver Hard Skin Remover Pedicure Tools with 3 